<div style='background-color:orange'>
<a id="TableOfContents"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            TABLE OF CONTENTS:
        </i></b></h1>
    <li><a href='#imports'>Imports</a>
    <li><a href='#orientation'>Orientation</a>
    <li><a href="#initial">Initial Setup</a>
    <li><a href='#testsingle'>Test With Single Column</a>
    <li><a href='#testall'>Test With All Columns</a>
    <li><a href='#functioncreation'>Create Function</a>
    <li><a href="#testfunction">Test Function</a>
    <li><a href='#misc'>Miscellaneous</a>
    </li>
</div>

<div style='background-color:orange'>
<a id="imports"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Imports
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# .py files
import wrangle as w

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

<div style='background-color:orange'>
<a id="orientation"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Orientation
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

This file is deliberately and solely for the purpose of creating a weighted risk aggregation score column for each row in order to determine if this score can reliably be utilized.

<h3><b><i>
    Basic Idea For Weighted Aggregation:
</i></b></h3>

1. FOR EACH COLUMN
    1. For each unique value, get the injury type distribution by percent
        - (UNIQUE: B%, A%, N%, K%)
    2. Multiply injury type percentage for each unique value
        - B(Minor injury) * 1
        - A(Major injury) * 2
        - N(No injury) * 0
        - K(Fatal injury) * 3
    3. Sum of values / 3 for each unique value in column to equal risk value
2. Create 'risk_agg' Column
    1. Sum of all scores / Total columns used
3. Visualize?

<div style='background-color:orange'>
<a id="initial"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Initial Setup
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [2]:
# Get the master filtered dataset
master = w.wrangle()
master.head(3)

,crash_id,$1000_damage_to_any_one_person's_property,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,adjusted_roadway_part,agency,at_intersection_flag,average_daily_traffic_amount,average_daily_traffic_year,...,charge,citation,person_age,person_ethnicity,person_gender,person_helmet,person_injury_severity,person_type,physical_location_of_an_occupant,crash_datetime
0,16189632,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,OPERATE UNREGISTERED MOTOR VEHICLE,TX52Q80UKZPL,37,W - WHITE,1 - MALE,1 - NOT WORN,A - SUSPECTED SERIOUS INJURY,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,1 - FRONT LEFT OR MOTORCYCLE DRIVER,2018-01-01 11:23:00
1,16203470,No,NO,no data,no data,1 - MAIN/PROPER LANE,"DEPARTMENT OF PUBLIC SAFETY, STATE OF TEXAS",False,no data,no data,...,"NO CLASS ""M"" LICENSE",TX52QD0NAP34,30,H - HISPANIC,1 - MALE,"3 - WORN, NOT DAMAGED",B - SUSPECTED MINOR INJURY,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,1 - FRONT LEFT OR MOTORCYCLE DRIVER,2018-01-04 13:16:00
2,16192023,Yes,NO,no data,no data,1 - MAIN/PROPER LANE,HARRIS COUNTY SHERIFF'S OFFICE,False,no data,no data,...,NO CHARGES,no data,21,W - WHITE,1 - MALE,"2 - WORN, DAMAGED",A - SUSPECTED SERIOUS INJURY,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE,1 - FRONT LEFT OR MOTORCYCLE DRIVER,2018-01-05 20:45:00


<div style='background-color:orange'>
<a id="testsingle"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Test With Single Column
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    <li><a href='#testsingletest'>Test</a>
    <li><a href='#testsinglefunction'>Function</a>
    <li><a href='#testsinglesummary'>Summarization</a>
    </li>
</div>

<a id='testsingletest'></a>
<h3><b><i>
    Test
</i></b></h3>
<li><a href='#testsingle'>Test With Single Column Top</a></li>

In [10]:
# Use a single column initially for testing purposes
master.vehicle_defect_1.value_counts()

vehicle_defect_1
no data                                13678
98 - OTHER  (EXPLAIN IN NARRATIVE)       187
12 - DEFECTIVE OR SLICK TIRES            152
10 - DEFECTIVE OR NO VEHICLE BRAKES      107
11 - DEFECTIVE STEERING MECHANISM         38
5 - DEFECTIVE OR NO HEADLAMPS             10
13 - DEFECTIVE TRAILER HITCH               1
9 - DEFECTIVE OR NO TRAILER BRAKES         1
Name: count, dtype: int64

In [12]:
# Create a smaller dataset with target and above column
cols_for_single_testing = [
    'vehicle_defect_1',
    'person_injury_severity'
]
single_test = master[cols_for_single_testing]
single_test.sample()

,vehicle_defect_1,person_injury_severity
6175,no data,A - SUSPECTED SERIOUS INJURY


In [63]:
# Testing with single unique value ('no data') in column
temp_dict = single_test[single_test.vehicle_defect_1 == 'no data']['person_injury_severity'].value_counts(normalize=True).to_dict()
for key in temp_dict.keys():
    if key == 'A - SUSPECTED SERIOUS INJURY':
        temp_dict['A - SUSPECTED SERIOUS INJURY'] = temp_dict['A - SUSPECTED SERIOUS INJURY'] * 2
    elif key == 'K - FATAL INJURY':
        temp_dict['K - FATAL INJURY'] = temp_dict['K - FATAL INJURY'] * 3
risk_score = sum(temp_dict.values()) / 3
print(risk_score)

0.4687088755666033


In [86]:
# Testing with all unique values to get risk scores
risk_dict = {
    'Unique Value' : [],
    'Risk Score' : []
}
for val in single_test.vehicle_defect_1.unique():
    injury_distribution_dict = single_test[single_test.vehicle_defect_1 == val]['person_injury_severity'].value_counts(normalize=True).to_dict()
    for key in injury_distribution_dict.keys():
        if key == 'A - SUSPECTED SERIOUS INJURY':
            injury_distribution_dict['A - SUSPECTED SERIOUS INJURY'] = injury_distribution_dict['A - SUSPECTED SERIOUS INJURY'] * 2
        elif key == 'K - FATAL INJURY':
            injury_distribution_dict['K - FATAL INJURY'] = injury_distribution_dict['K - FATAL INJURY'] * 3
    risk_score = sum(injury_distribution_dict.values()) / 3
    risk_dict['Unique Value'].append(val)
    risk_dict['Risk Score'].append(risk_score)
pd.DataFrame(risk_dict)

,Unique Value,Risk Score
0,no data,0.468709
1,10 - DEFECTIVE OR NO VEHICLE BRAKES,0.386293
2,98 - OTHER (EXPLAIN IN NARRATIVE),0.445633
3,12 - DEFECTIVE OR SLICK TIRES,0.467105
4,11 - DEFECTIVE STEERING MECHANISM,0.421053
5,5 - DEFECTIVE OR NO HEADLAMPS,0.600000
6,13 - DEFECTIVE TRAILER HITCH,0.333333
7,9 - DEFECTIVE OR NO TRAILER BRAKES,0.333333


In [81]:
# Match value existing in dataset with risk_dict to get risk_score
for val in single_test.vehicle_defect_1:
    risk_index = risk_dict['Unique Value'].index(val)
    risk = risk_dict['Risk Score'][risk_index]
    print(risk)

0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.3862928348909657
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4687088755666033
0.4456327985

In [92]:
# Put it all together :D
risk_dict = {
    'Unique Value' : [],
    'Risk Score' : []
}
for unique_value in single_test.vehicle_defect_1.unique():
    injury_distribution_dict = single_test[single_test.vehicle_defect_1 == unique_value]['person_injury_severity'].value_counts(normalize=True).to_dict()
    for injury_type in injury_distribution_dict.keys():
        if injury_type == 'A - SUSPECTED SERIOUS INJURY':
            injury_distribution_dict['A - SUSPECTED SERIOUS INJURY'] = injury_distribution_dict['A - SUSPECTED SERIOUS INJURY'] * 2
        elif injury_type == 'K - FATAL INJURY':
            injury_distribution_dict['K - FATAL INJURY'] = injury_distribution_dict['K - FATAL INJURY'] * 3
    risk_score = sum(injury_distribution_dict.values()) / 3
    risk_dict['Unique Value'].append(unique_value)
    risk_dict['Risk Score'].append(risk_score)
risk_value_list = []
for value in single_test.vehicle_defect_1:
    risk_index = risk_dict['Unique Value'].index(value)
    risk = risk_dict['Risk Score'][risk_index]
    risk_value_list.append(risk)
single_test['risk_score'] = risk_value_list
single_test

,vehicle_defect_1,person_injury_severity,risk_score
0,no data,A - SUSPECTED SERIOUS INJURY,0.468709
1,no data,B - SUSPECTED MINOR INJURY,0.468709
2,no data,A - SUSPECTED SERIOUS INJURY,0.468709
3,no data,B - SUSPECTED MINOR INJURY,0.468709
4,no data,B - SUSPECTED MINOR INJURY,0.468709
...,...,...,...
14169,no data,B - SUSPECTED MINOR INJURY,0.468709
14170,5 - DEFECTIVE OR NO HEADLAMPS,A - SUSPECTED SERIOUS INJURY,0.600000
14171,no data,A - SUSPECTED SERIOUS INJURY,0.468709
14172,no data,B - SUSPECTED MINOR INJURY,0.468709


In [93]:
# Verify vehicle_defect_1 with risk_score
single_test.groupby('vehicle_defect_1')['risk_score'].value_counts()

vehicle_defect_1                     risk_score
10 - DEFECTIVE OR NO VEHICLE BRAKES  0.386293        107
11 - DEFECTIVE STEERING MECHANISM    0.421053         38
12 - DEFECTIVE OR SLICK TIRES        0.467105        152
13 - DEFECTIVE TRAILER HITCH         0.333333          1
5 - DEFECTIVE OR NO HEADLAMPS        0.600000         10
9 - DEFECTIVE OR NO TRAILER BRAKES   0.333333          1
98 - OTHER  (EXPLAIN IN NARRATIVE)   0.445633        187
no data                              0.468709      13678
Name: count, dtype: int64

---

<a id='testsinglefunction'></a>
<h3><b><i>
    Function
</i></b></h3>
<li><a href='#testsingle'>Test With Single Column Top</a></li>

In [95]:
def risk_scores_single_column(df, col_name):
    '''
    Takes in a pandas dataframe and the specific column that you want a specific
    risk value for each specific value in the column...
    Gives a list of risk values associated by each value in the column as well
    as the dictionary of unique values and their risk value...
    
    INPUT:
    df = Pandas dataframe for utilization
    col_name = The exact column name within 'df' that you want
    
    OUTPUT:
    risk_value_list = List of risk values by each value in the column
    '''
    risk_dict = {
    'Unique Value' : [],
    'Risk Score' : []
    }
    for unique_value in df.col_name.unique():
        injury_distribution_dict = df[df.col_name == unique_value]['person_injury_severity'].value_counts(normalize=True).to_dict()
        for injury_type in injury_distribution_dict.keys():
            if injury_type == 'A - SUSPECTED SERIOUS INJURY':
                injury_distribution_dict['A - SUSPECTED SERIOUS INJURY'] = injury_distribution_dict['A - SUSPECTED SERIOUS INJURY'] * 2
            elif injury_type == 'K - FATAL INJURY':
                injury_distribution_dict['K - FATAL INJURY'] = injury_distribution_dict['K - FATAL INJURY'] * 3
        risk_score = sum(injury_distribution_dict.values()) / 3
        risk_dict['Unique Value'].append(unique_value)
        risk_dict['Risk Score'].append(risk_score)
    risk_value_list = []
    for value in df.col_name:
        risk_index = risk_dict['Unique Value'].index(value)
        risk = risk_dict['Risk Score'][risk_index]
        risk_value_list.append(risk)
    return risk_dict, risk_value_list

In [100]:
# Test the function
test_dict, test_list = risk_scores_single_column(single_test, 'vehicle_defect_1')
print(pd.DataFrame(test_dict))
print(test_list)

                          Unique Value  Risk Score
0                              no data    0.468709
1  10 - DEFECTIVE OR NO VEHICLE BRAKES    0.386293
2   98 - OTHER  (EXPLAIN IN NARRATIVE)    0.445633
3        12 - DEFECTIVE OR SLICK TIRES    0.467105
4    11 - DEFECTIVE STEERING MECHANISM    0.421053
5        5 - DEFECTIVE OR NO HEADLAMPS    0.600000
6         13 - DEFECTIVE TRAILER HITCH    0.333333
7   9 - DEFECTIVE OR NO TRAILER BRAKES    0.333333
[0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.3862928348909657, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 0.4687088755666033, 

---

<a id='testsinglesummary'></a>
<h3><b><i>
    Summarization
</i></b></h3>
<li><a href='#testsingle'>Test With Single Column Top</a></li>

1. <b>Create risk_dict // PURPOSE: Assign each value a risk score</b>
    - Each unique value is a key
    - Risk score is value
2. <b>Create risk_value_list // PURPOSE: Risk score by value in row</b>
    - Using value in row, find index in risk_dict
    - Using index, get risk score

<div style='background-color:orange'>
<a id="testall"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Test With All Columns
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    <li><a href='#testalltest'>Test</a>
    <li><a href='#testallfunction'>Function</a>
    <li><a href='#testallsummary'>Summarization</a>
    </li>
</div>

<a id='testalltest'></a>
<h3><b><i>
    Test
</i></b></h3>
<li><a href='#testall'>Test With Single Column Top</a></li>

---

<a id='testallfunction'></a>
<h3><b><i>
    Function
</i></b></h3>
<li><a href='#testall'>Test With Single Column Top</a></li>

---

<a id='testallsummary'></a>
<h3><b><i>
    Summarization
</i></b></h3>
<li><a href='#testall'>Test With Single Column Top</a></li>

<div style='background-color:orange'>
<a id="functioncreation"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Create Function
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

<div style='background-color:orange'>
<a id="testfunction"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Test Function
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

<div style='background-color:orange'>
<a id="misc"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Miscellaneous
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>